In [6]:
import pandas as pd

In [10]:
path = "salaries.csv"
data = pd.read_csv(path,delimiter='\t',on_bad_lines='skip')
print(data.head())
print(data.info())

             role country       company  company_rating       job_title  \
0  Data Scientist   Japan           CTW             2.9  Data Scientist   
1  Data Scientist   Japan      FUJIFILM             3.7  Data Scientist   
2  Data Scientist   Japan  GMO Internet             3.3  Data Scientist   
3  Data Scientist   Japan      Data4Cʼs             3.0  Data Scientist   
4  Data Scientist   Japan      Deloitte             4.0  Data Scientist   

       median    low   high   unit  Unnamed: 9  
0  ¥1,500,000    ¥1M   ¥18M   / yr         NaN  
1  ¥5,000,000    ¥5M    ¥7M   / yr         NaN  
2  ¥6,500,000    ¥7M     -    / yr         NaN  
3    ¥350,000  ¥300K  ¥402K   / mo         NaN  
4  ¥6,250,000    ¥5M    ¥8M   / yr         NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401190 entries, 0 to 401189
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   role            401190 non-null  object 
 1   c

In [12]:
data = data.drop(columns=['Unnamed: 9'])

In [13]:
import csv

In [14]:
file_path = "salaries.csv"
with open(file_path,'r',encoding='utf-8') as file:
    reader = csv.reader(file,delimiter='\t')
    for i, row in enumerate(reader):
        if len(row) != 9:
            print(f'Bad line: {i+1}, Content: {row}') 

Bad line: 1, Content: ['role', 'country', 'company', 'company_rating', 'job_title', 'median', 'low', 'high', 'unit', '']


In [17]:
data = pd.read_csv("salaries.csv",header=0,delimiter='\t')
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
data = data.replace('\xa0', ' ', regex=True)
data = data.drop(columns=['Unnamed: 9'])
mean_rating = data['company_rating'].mean()
data['company_rating'].fillna(mean_rating, inplace=True)
if 'company' in data.columns:
    data['company'] = data['company'].fillna("No Company")

In [19]:
def Changing_units(unit):
    if unit == ' / yr':
        return "Yearly"
    elif unit == ' / mo':
        return "Monthly"
    elif unit == ' / hr':
        return "Hourly"


data['unit'] = data['unit'].apply(Changing_units)
    

In [20]:
def clmn_currency(val):
    units = val.split(' ')
    if len(units) == 1:
        return val[0]
    else:
        return units[0]
data['currency'] = data['median'].apply(clmn_currency)

In [21]:
def format_salary(x):
    x = str(x)
    unit = clmn_currency(x)
    x = x.replace(unit, '')
    x = x.replace(',', '')
    x = x.replace('M', '000000')
    x = x.replace('K', '000')
    return x


data['median'] = data['median'].apply(format_salary)
data['low'] = data['low'].apply(format_salary)
data['high'] = data['high'].apply(format_salary)

In [22]:
data.to_csv('cleaned_salaries.csv', index=False)

In [23]:
pip install pyodbc

   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.7 kB ? eta -:--:--
   ----------------------------------- ---- 61.4/68.7 kB 3.2 MB/s eta 0:00:01
   ---------


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import pandas as pd
import pyodbc

# Connection string for your SQL Server database
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=AHMAR;"
    "DATABASE=Salaries;"  # Assuming the database 'Salaries' already exists
    "Trusted_Connection=yes;"
)

# Path to your CSV file
csv_file_path = 'D:\\Python+SQL+PowerBi\\cleaned_salaries.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Connect to the SQL Server database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Is baat ka lazmi dehan rakhna hai k tables database mei exist krna chieyeee
create_table_query = '''
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'SALARIES')
BEGIN
    CREATE TABLE SALARIES (
        role VARCHAR(255),
        country VARCHAR(255),
        company VARCHAR(255),
        company_rating FLOAT,
        job_title VARCHAR(255),
        median VARCHAR(255),
        low VARCHAR(255),
        high VARCHAR(255),
        unit VARCHAR(255),
        currency VARCHAR(255)
    );
END
'''
cursor.execute(create_table_query)
conn.commit()

# Tables mei data insert horaha hai is step mei
for index, row in df.iterrows():
    insert_query = '''
    INSERT INTO SALARIES (role, country, company, company_rating, job_title, median, low, high, unit, currency)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.execute(insert_query, row['role'], row['country'], row['company'], row['company_rating'],
                   row['job_title'], row['median'], row['low'], row['high'], row['unit'], row['currency'])

conn.commit()

# Best practice hoti hai cursor or connection dono ko hamesha close karna
cursor.close()
conn.close()

print("Data loaded successfully!")

Data loaded successfully!
